### USANDO SLQALCHEMY
#### Etapa 1.Conexão com o banco de dados e obtenção dos metadados
Essa etapa irá se conectar ao banco de dados PostgreSQL e obter os metadados de todas as tabelas.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Carregar variáveis do arquivo .env
load_dotenv()

# Função para conectar ao banco de dados usando SQLAlchemy
def connect_db_sqlalchemy():
    try:
        # Pegar as credenciais do arquivo .env
        db_host = os.getenv('DB_HOST')
        db_database = os.getenv('DB_DATABASE')
        db_user = os.getenv('DB_USER')
        db_password = os.getenv('DB_PASSWORD')
        db_port = os.getenv('DB_PORT')

        # Criar a URL de conexão no formato aceito pelo SQLAlchemy
        db_url = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}'
        
        # Criar o engine de conexão usando SQLAlchemy
        engine = create_engine(db_url)
        print("Conexão com o banco de dados estabelecida com sucesso via SQLAlchemy!")
        return engine
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados com SQLAlchemy: {e}")
        return None

# Função para obter os metadados de todas as tabelas usando SQLAlchemy
def get_all_table_metadata_sqlalchemy(engine):
    try:
        query = """
        SELECT table_name, column_name, data_type 
        FROM information_schema.columns 
        WHERE table_schema = 'public'
        ORDER BY table_name;
        """
        metadata_df = pd.read_sql_query(query, engine)
        print("Metadados obtidos com sucesso via SQLAlchemy!")
        return metadata_df
    except Exception as e:
        print(f"Erro ao obter metadados via SQLAlchemy: {e}")
        return None

# Testar a conexão e obtenção de metadados com SQLAlchemy
engine = connect_db_sqlalchemy()

if engine:
    metadata_df = get_all_table_metadata_sqlalchemy(engine)
    if metadata_df is not None:
        print(metadata_df.head())  # Mostrando uma amostra dos metadados obtidos
    else:
        print("Erro ao carregar metadados.")
else:
    print("Erro na conexão com o banco de dados via SQLAlchemy.")


In [ ]:
metadata_df


### Passo 2. Gerando contexto
Consulta aprimorada: Estamos agora capturando mais informações sobre as colunas, incluindo:

1. table_name: Nome da tabela.
2. column_name: Nome da coluna.
3. data_type: Tipo de dado da coluna (ex. varchar, integer).
4. is_nullable: Informa se a coluna pode ser nula (YES ou NO).
5. character_maximum_length: Limite de caracteres (se for do tipo varchar ou outros tipos de texto).
6. Organização dos resultados: O resultado é organizado em um dataframe pandas, com as informações sobre as tabelas e colunas do banco de dados.
7. Contagem de tabelas: O código também imprime a quantidade de tabelas únicas no banco de dados, utilizando o método nunique() do pandas para contar as tabelas.
8. Debug visual: exibe as 10 primeiras linhas dos metadados para que você possa verificar visualmente se os dados foram carregados corretamente.


---MELHORIAS ---
1. TRATAR ERROS DE CONSULTA
2. MELHORAR O PROMPT PARA TRAZER MAIOR ASSERTIVIDADE NAS CONSULTAS 

    2.1 Adicionar composiçções para melhorar os selects, ex: em buscas de nomes fazer query usadno like = '%{valor}%'

3. ADICIONAR OS RELACIONAMENTOS E CHAVES USADAS NO BANCO
4. ADICIONAR FUNCIONALIDADE DO WHISPER MANTER CONVERSACAO POR VOZ


### USANDO SLQALCHEMY
#### Etapa 1.Conexão com o banco de dados e obtenção dos metadados
Essa etapa irá se conectar ao banco de dados PostgreSQL e obter os metadados de todas as tabelas.

In [ ]:
from sqlalchemy import text

# Função para obter os metadados de todas as tabelas usando SQLAlchemy
def get_all_table_metadata_sqlalchemy(engine):
    try:
        # Consulta para buscar nome da tabela, nome da coluna, tipo de dado, is_nullable e comprimento máximo de caracteres
        query = text("""
        SELECT table_name, column_name, data_type, is_nullable, character_maximum_length
        FROM information_schema.columns 
        WHERE table_schema = 'public'
        ORDER BY table_name, ordinal_position;
        """)
        
        # Executar a consulta e obter o resultado em um dataframe
        metadata_df = pd.read_sql_query(query, engine)
        print("Metadados obtidos com sucesso via SQLAlchemy!")
        
        # Exibir algumas linhas para verificar se os dados estão corretos
        print(metadata_df.head())
        
        # Exibir uma visão geral do número de tabelas
        table_count = metadata_df['table_name'].nunique()

        print(f"O banco de dados contém {table_count} tabelas.")
        #metadata_df['table_name'] = metadata_df['table_name'].apply(lambda x: f'"{x}"')
        # Verificação e ajuste para nomes de colunas com aspas
        # Força a manter os nomes exatamente como estão no banco
        metadata_df['column_name'] = metadata_df['column_name'].apply(lambda x: f'"{x}"' if x != x.lower() else x)
        metadata_df['table_name'] = metadata_df['table_name'].apply(lambda x: f'"{x}"' if x != x.lower() else x)
        
        return metadata_df
    except Exception as e:
        print(f"Erro ao obter metadados via SQLAlchemy: {e}")
        return None

# Testar a função de obtenção de metadados com SQLAlchemy
if engine:
    metadata_df = get_all_table_metadata_sqlalchemy(engine)
    if metadata_df is not None:
        # Visualizar o dataframe com os metadados corrigidos
        print(metadata_df.head(10))  # Mostrar as 10 primeiras linhas para ver as colunas com nomes entre aspas
    else:
        print("Erro ao carregar metadados.")
else:
    print("Erro na conexão com o banco de dados via SQLAlchemy.")


In [ ]:
metadata_df


### Elaborar a query

In [ ]:
# Importações necessárias
import openai
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# Carregar as variáveis do .env, incluindo a chave da API do OpenAI
load_dotenv()

# Função para carregar a chave da API OpenAI do .env
def get_openai_api_key():
    try:
        openai_api_key = os.getenv('OPENAI_API_KEY')
        if not openai_api_key:
            raise ValueError("A chave da API OpenAI não foi encontrada no arquivo .env.")
        print("Chave da API OpenAI carregada com sucesso.")
        return openai_api_key
    except Exception as e:
        print(f"Erro ao carregar a chave da API: {e}")
        return None

# Função para gerar uma consulta SQL a partir de uma pergunta usando LangChain e OpenAI
def generate_sql_from_question(pergunta, contexto):
    try:
        # Carregar a chave da API OpenAI do .env
        openai_api_key = get_openai_api_key()
        if not openai_api_key:
            return None

        # Configurar a API key para o OpenAI
        openai.api_key = openai_api_key

        # Definir o template do prompt que será passado para o LLM
        prompt_template = """
        Você é um assistente de banco de dados que entende a estrutura do banco de dados PostgresSQl, consegue interpretar e traduzir de linguagem natural para SQL as entradas do usuario. O usuário fez a seguinte pergunta:

        Pergunta: {pergunta}

        Baseado nesses metadados, escreva uma consulta SQL para responder à seguinte pergunta:

        Aqui estão os metadados do banco de dados:
        {contexto_tabela}

        Gere uma consulta SQL com base na pergunta do usuário, levando em consideração as colunas sensíveis a maiúsculas e minúsculas e as aspas duplas apropriadas.
        **Importante**: Quando a pergunta do usuário envolver buscas textuais (ex.: bairros, nomes), sempre utilize `ILIKE '%valor%'` para garantir que a busca seja por qualquer parte do texto que contenha o valor buscado.
        SQL:
        """

        # Criar o template do LangChain
        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["contexto_tabela", "pergunta"]
        )

        # Inicializar o modelo OpenAI para LangChain
        llm = OpenAI()

        # Configurar a cadeia de LangChain para gerar a query SQL
        chain = LLMChain(llm=llm, prompt=prompt)

        # Gerar a consulta SQL com base na pergunta do usuário e nos metadados
        sql_query = chain.run(contexto_tabela=contexto, pergunta=pergunta)
        print("Consulta SQL gerada com sucesso!")
        return sql_query
    except Exception as e:
        print(f"Erro ao gerar a consulta SQL: {e}")
        return None

# Exemplo de pergunta do usuário e contexto dos metadados gerados anteriormente
pergunta_usuario = "quantos imóveis existem cadastrados na base no bairro Manaira"

# Gerar uma amostra de contexto baseado nos metadados obtidos (você pode expandir isso conforme necessário)
if metadata_df is not None:
    contexto_banco = metadata_df.to_string(index=False)

    # Gerar a consulta SQL usando a pergunta do usuário e o contexto dos metadados
    sql_query_gerada = generate_sql_from_question(pergunta_usuario, contexto_banco)

    # Mostrar a consulta SQL gerada
    if sql_query_gerada:
        print(f"Consulta SQL gerada:\n{sql_query_gerada}")
    else:
        print("Erro ao gerar a consulta SQL.")
else:
    print("Os metadados do banco de dados não estão disponíveis.")


In [ ]:
result = session.query(Tabela).filter(Tabela.campo.ilike('%Manaira%')).all()


In [ ]:
print(sql_query_gerada)
sql_query_gerada = "SELECT COUNT(*) FROM imoveis WHERE desc_bair ILIKE '%Manaira%';"
print(sql_query_gerada)
sql_query_gerada=sql_query_gerada.strip()
print(sql_query_gerada)


In [ ]:
#print(f"Consulta SQL gerada:\n{sql_query_gerada}")

# Função para executar a consulta SQL gerada no banco de dados
def execute_query(engine, query):
    try:
        # Executar a consulta SQL e retornar o resultado em um dataframe pandas
        df_result = pd.read_sql_query(query, engine)
        print("Consulta SQL executada com sucesso!")
        return df_result
    except Exception as e:
        print(f"Erro ao executar a consulta SQL: {e}")
        return None

# Verificar se a consulta SQL foi gerada corretamente
if sql_query_gerada:
    # Executar a consulta SQL gerada no banco de dados
    df_resultado = execute_query(engine, sql_query_gerada)

    # Exibir os resultados obtidos, se houver algum
    if df_resultado is not None:
        print("Resultado da consulta:")
        print(df_resultado.head())  # Mostra as primeiras linhas do resultado
    else:
        print("Nenhum resultado encontrado ou erro na execução da consulta.")
else:
    print("Nenhuma consulta SQL foi gerada.")


In [ ]:
# Função para gerar o prompt de interpretação de resultados
def interpret_result_with_template(df_resultado, pergunta_usuario, sql_query_gerada):
    try:
        # Transformar o dataframe em string para que o modelo possa interpretar
        result_str = df_resultado.to_string(index=False)
        
        # Criar um template de prompt para LangChain interpretar os resultados
        prompt_template = """
        Você é um assistente de banco de dados. O usuário fez a seguinte pergunta:

        Pergunta: {pergunta}

        Você gerou a seguinte consulta SQL com base nos metadados do banco de dados:

        SQL: {sql_query}

        A consulta SQL retornou os seguintes dados:

        {resultado}

        Com base nisso, explique o resultado da consulta de forma clara e amigável e direta para o usuário.
        """

        # Criar o template do LangChain para processar o prompt
        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["pergunta", "sql_query", "resultado"]
        )

        # Inicializar o modelo OpenAI para LangChain
        llm = OpenAI()
        
        # Criar a cadeia LangChain para gerar a interpretação da resposta
        chain = LLMChain(llm=llm, prompt=prompt)

        # Gerar a resposta interpretada em linguagem natural
        resposta_interpretada = chain.run(pergunta=pergunta_usuario, sql_query=sql_query_gerada, resultado=result_str)
        return resposta_interpretada
    except Exception as e:
        print(f"Erro ao interpretar o resultado: {e}")
        return "Houve um erro ao interpretar os resultados."

# Exemplo de execução da função de interpretação de resultados
if df_resultado is not None and sql_query_gerada:
    resposta_interpretada = interpret_result_with_template(df_resultado, pergunta_usuario, sql_query_gerada)
    print(f"Resposta interpretada em linguagem natural: {resposta_interpretada}")
else:
    print("Não há resultados ou consulta SQL para interpretar.")


# NOVAS FUNCIONALIDADES

DADOS RELACIONAMENTO

In [ ]:
from sqlalchemy import inspect

# Função para obter o contexto detalhado do banco de dados, incluindo relações entre as tabelas
def get_database_context(engine):
    try:
        inspector = inspect(engine)
        table_context = {}
        
        # Obter todas as tabelas no banco de dados
        tables = inspector.get_table_names()
        
        # Iterar sobre as tabelas para obter suas colunas, chaves primárias e estrangeiras
        for table in tables:
            columns = inspector.get_columns(table)
            primary_key = inspector.get_pk_constraint(table)
            foreign_keys = inspector.get_foreign_keys(table)
            
            table_context[table] = {
                "columns": columns,
                "primary_key": primary_key,
                "foreign_keys": foreign_keys
            }
        
        print("Contexto das tabelas e relações capturado com sucesso!")
        return table_context
    except Exception as e:
        print(f"Erro ao obter o contexto do banco de dados: {e}")
        return None

# Função para formatar o contexto do banco de dados para o agente
def format_database_context(table_context):
    context_str = "Estrutura do banco de dados:\n"
    
    for table, details in table_context.items():
        context_str += f"\nTabela: {table}\n"
        context_str += "Colunas:\n"
        for column in details['columns']:
            context_str += f"- {column['name']} ({column['type']})\n"
        
        context_str += "Chave Primária:\n"
        context_str += f"- {details['primary_key']['constrained_columns']}\n" if details['primary_key'] else "Nenhuma\n"
        
        context_str += "Chaves Estrangeiras:\n"
        if details['foreign_keys']:
            for fk in details['foreign_keys']:
                context_str += f"- {fk['constrained_columns']} referenciando {fk['referred_table']}.{fk['referred_columns']}\n"
        else:
            context_str += "Nenhuma\n"
    
    return context_str

# Testar a captura do contexto do banco de dados
if engine:
    table_context = get_database_context(engine)
    if table_context:
        contexto_formatado = format_database_context(table_context)
        print(contexto_formatado)
    else:
        print("Erro ao obter o contexto detalhado do banco de dados.")
else:
    print("Erro na conexão com o banco de dados via SQLAlchemy.")
